In [1]:
import numpy as np
import pandas as pd
import os
import json

num_trails = 10
cwd = os.getcwd()

results_path = cwd[:-28] + "Results/T5_finetuned/all_scores_range_0_138_trail_{}.npz"

results_intent_path = cwd[:-28] + "Results/T5_finetuned/all_scores_range_by_intent_0_138_trail_{}.txt"

intents = ["What is the state's policy regarding education?",
           "What are the major historical events in this state?",
           "How is the weather of the state?",
           "Which places seem interesting to you for visiting in this state?",
           "What are some of the most interesting things about this state?"]

sample_size = 10

model = "T5 Finetune"


In [2]:
full_results = []
total_data = None
total_data_intent = {}

for intent in intents:
    total_data_intent[intent] = []

# get all of the "normal" data
for i in range(sample_size):
    temp = np.load(results_path.format(i))
    data = temp['scores']
    
    if total_data is None:
        # we need to take the mean of axis-one first because each "user-sample" contains a set of predicted summaries. So we first avg those sets, before average across all the data/trials
        total_data = data.mean(axis=1)
    else:
        total_data = np.concatenate((total_data, data.mean(axis=1)), axis=0)

# get all of the "by-intent" data
for i in range(sample_size):
    res = None
    with open(results_intent_path.format(i)) as f:
        res = json.load(f)

    for intent in intents:
        total_data_intent[intent].extend(np.mean(res[intent], axis=1).tolist())

# now calculate the relavent stats
total_mean = np.mean(total_data, axis=0)
total_stddev = np.std(total_data, axis=0)

mean_intent = {}
stddev_intent = {}

for intent in intents:
    mean = np.mean(total_data_intent[intent], axis=0)
    stddev = np.std(total_data_intent[intent], axis=0)
    mean_intent[intent] = mean
    stddev_intent[intent] = stddev

In [3]:
metrics = ["Rouge-1", "Rouge-2", "Rouge-L"]

for i, (row_m, row_s) in enumerate(zip(total_mean, total_stddev)):
    row_str = model + " & "
    for col_m, col_s in zip(row_m, row_s):
        row_str += str(np.round(col_m, 4)) + " $\\pm$ " + str(np.round(col_s, 4)) + " & "
    
    row_str = row_str[:-3]
    row_str += " \\\\"
    print(metrics[i])
    print(row_str)

Rouge-1
T5 Finetune & 0.1257 $\pm$ 0.0365 & 0.5222 $\pm$ 0.1124 & 0.1949 $\pm$ 0.0452 & 0.1218 $\pm$ 0.0282 \\
Rouge-2
T5 Finetune & 0.0334 $\pm$ 0.0227 & 0.1543 $\pm$ 0.1306 & 0.0529 $\pm$ 0.0367 & 0.033 $\pm$ 0.023 \\
Rouge-L
T5 Finetune & 0.0871 $\pm$ 0.0253 & 0.3698 $\pm$ 0.1062 & 0.1356 $\pm$ 0.0334 & 0.0847 $\pm$ 0.0209 \\


In [4]:
metrics = ["Rouge-1", "Rouge-2", "Rouge-L"]

for intent in intents:
    print("\nResults for intent {}\n".format(intent))
    for i, (row_m, row_s) in enumerate(zip(mean_intent[intent], stddev_intent[intent])):
        row_str = "\t"+model + " & "
        for col_m, col_s in zip(row_m, row_s):
            row_str += str(np.round(col_m, 4)) + " $\\pm$ " + str(np.round(col_s, 4)) + " & "
        
        row_str = row_str[:-3]
        row_str += " \\\\"
        print("\t"+metrics[i])
        print(row_str)


Results for intent What is the state's policy regarding education?

	Rouge-1
	T5 Finetune & 0.1408 $\pm$ 0.0309 & 0.4482 $\pm$ 0.0602 & 0.2081 $\pm$ 0.0349 & 0.13 $\pm$ 0.0218 \\
	Rouge-2
	T5 Finetune & 0.0267 $\pm$ 0.0098 & 0.0897 $\pm$ 0.0344 & 0.0399 $\pm$ 0.0139 & 0.025 $\pm$ 0.0087 \\
	Rouge-L
	T5 Finetune & 0.097 $\pm$ 0.02 & 0.3134 $\pm$ 0.0448 & 0.1437 $\pm$ 0.0222 & 0.0898 $\pm$ 0.0139 \\

Results for intent What are the major historical events in this state?

	Rouge-1
	T5 Finetune & 0.1041 $\pm$ 0.0328 & 0.5264 $\pm$ 0.0807 & 0.1675 $\pm$ 0.0407 & 0.1047 $\pm$ 0.0255 \\
	Rouge-2
	T5 Finetune & 0.0252 $\pm$ 0.0133 & 0.1328 $\pm$ 0.0548 & 0.0408 $\pm$ 0.0188 & 0.0255 $\pm$ 0.0117 \\
	Rouge-L
	T5 Finetune & 0.0721 $\pm$ 0.0213 & 0.3703 $\pm$ 0.06 & 0.1163 $\pm$ 0.026 & 0.0727 $\pm$ 0.0163 \\

Results for intent How is the weather of the state?

	Rouge-1
	T5 Finetune & 0.1254 $\pm$ 0.0289 & 0.5104 $\pm$ 0.0733 & 0.1937 $\pm$ 0.0334 & 0.1211 $\pm$ 0.0209 \\
	Rouge-2
	T5 Finetune 